In [ ]:
%load_ext autoreload
%autoreload 2

In [1]:
import sys
import os

# Add the path to the folder containing ClusterAndPredict
module_path = os.path.abspath(os.path.join('..', '/workspaces/FactChecker-V3'))
if module_path not in sys.path:
    sys.path.append(module_path)

import pandas as pd
from ClusterAndPredict.ClusterAndPredict import ClusterAndPredict
from Testing.DataLoader import DataLoader
from Testing.ParameterCreator import ParameterCreator
from Clustering.Helpers.Visualizer import Visualizer

2024-11-15 20:03:56.095110: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 20:03:56.097903: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-11-15 20:03:56.106701: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-15 20:03:56.121156: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-15 20:03:56.125332: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-15 20:03:56.137486: I tensorflow/core/platform/cpu_feature_gu

In [18]:
params = ParameterCreator().get_parameters()
results = []
cluster_dfs = []
print("Number of experiments to run: ", len(params))
for param in params:
    loc_results = pd.read_csv('/workspaces/FactChecker-V3/Testing/Test Output/Iterations/cluster_df_40.csv')
    # Filter results to where predicted_veracity is 4
    loc_results = loc_results[loc_results['predicted_veracity'] == 4]
    # Rename 'Text' column to 'text'
    loc_results = loc_results.rename(columns={'Text': 'text', 'veracity': 'Numerical Rating'})
    percentage = 0.75
    data_loader = DataLoader(percentage, True, param['random_seed'])
    use_only_card = param['use_only_CARD']
    size_of_dataset = param['size_of_dataset']
    del param['size_of_dataset']
    del param['use_only_CARD']

    train_df, test_df = data_loader.create_train_test_df(False, False, False, True, size_of_dataset)
    if use_only_card:
        print("using card")
        train_df, test_df = data_loader.create_train_test_df(True, True, True, True, size_of_dataset)

    # Filter train_df and test_df to where 'text' is in results['text']
    train_df = train_df[train_df['Text'].isin(loc_results['text'])]
    test_df = test_df[test_df['Text'].isin(loc_results['text'])]


    clf = ClusterAndPredict(**param, train_df=train_df)
    clf.fit(test_df['Text'].tolist(), test_df['Numerical Rating'].tolist())
    # Print best parameters
    best_estimator = clf
    score = best_estimator.score([], [])
    object_output = best_estimator.get_all_performance_metrics()
    cluster_df = object_output['cluster_df']
    cluster_dfs.append(cluster_df)
    output = {
        'percentage': percentage,
        'score': score,
        'accuracy': best_estimator.get_accuracy(),
        'was_supervised_umap_used': best_estimator.get_was_supervised(),
        'metrics': best_estimator.get_all_performance_metrics(),
        'size_of_dataset': 1.0,
        'use_only_CARD': False
    }
    # Prepend the value 'param' to the keys in params
    for key, value in param.items():
        local_key = 'params.' + key
        output[local_key] = value

    results.append(output)
    results_df = pd.json_normalize(results)
    results = []


All data sources are being used
Number of experiments to run:  1
Numerical Rating
3    3256
1      28
Name: count, dtype: int64
Numerical Rating
3    771
1     51
Name: count, dtype: int64
using card
All data sources are being used
Numerical Rating
3    4412
1    2474
Name: count, dtype: int64
Numerical Rating
3    893
1    828
Name: count, dtype: int64
Fitting
Getting embeddings...
Getting embeddings for batch  0  out of  508
Getting embeddings for batch  50  out of  508
Getting embeddings for batch  100  out of  508
Getting embeddings for batch  150  out of  508
Getting embeddings for batch  200  out of  508
Getting embeddings for batch  250  out of  508
Getting embeddings for batch  300  out of  508
Getting embeddings for batch  350  out of  508
Getting embeddings for batch  400  out of  508
Getting embeddings for batch  450  out of  508
Getting embeddings for batch  500  out of  508
Getting embeddings for batch  0  out of  2360
Getting embeddings for batch  50  out of  2360
Getting

/home/codespace/.local/lib/python3.10/site-packages/umap/umap_.py:2437: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


Running hdbscan...
breaking further...
93
95
97
97
99
101
101
103
103
103
103
103
103
103
105
105
107
107
109
109
113
113
113
113
113
116
116
116
116
116
116
116
116
116
118
118
118
118
118
118
118
118
118
118
cluster  predict  veracity
-1       False     3          712
                   1          538
         True     -1          330
 0       False     3            3
         True     -1            2
                             ... 
 116     True     -1            2
 117     False     3            5
                   1            2
 118     False     3            7
                   1            1
Name: count, Length: 163, dtype: int64
breaking further...
118
120
122
122
122
122
122
122
122
122
122
122
122
cluster  predict  veracity
-1       False     3          796
                   1          570
         True     -1          372
 0       False     3            3
         True     -1            2
                             ... 
 120     True     -1            2
 121     Fals

In [19]:
print(len(train_df))
print(len(test_df))

2360
508


In [21]:
# PRint value counts for results['predicted_veracity']
results = pd.read_csv('/workspaces/FactChecker-V3/Testing/Test Output/Iterations/cluster_df_40.csv')
print(loc_results['predicted_veracity'].value_counts())

predicted_veracity
4    2568
Name: count, dtype: int64


In [20]:
output

{'percentage': 0.75,
 'score': 1.9746835443037973,
 'accuracy': 0.23228346456692914,
 'was_supervised_umap_used': False,
 'metrics': {'accuracy': 0.23228346456692914,
  'accuracy_not_including_fours': 0.9147286821705426,
  'percentage_of_fours': 0.7460629921259843,
  'percentage_of_no_clusters_in_ground_truth': 0.0,
  'precision_on_three': 1.0,
  'recall_on_three': 0.26101694915254237,
  'average_confidence_for_3': 0,
  'precision_on_three_excluding_fours': 1.0,
  'recall_on_three_excluding_fours': 0.9746835443037974,
  'precision_on_one': 1.0,
  'recall_on_one': 0.19248826291079812,
  'precision_on_one_excluding_fours': 1.0,
  'recall_on_one_excluding_fours': 0.82,
  'precision': 0.2459508785758667,
  'recall': 0.23228346456692914,
  'precision_no_fours': 0.917883540652605,
  'recall_no_fours': 0.9147286821705426,
  'cluster_df':                                                    text  veracity  predict  \
  0     The authors find sea surface temperatures vari...         1     True   